In [ ]:
# Julia version code
"""
nonnegative linear regression 
"""

# Notes
# 1. Note we set min A_ij = 1. This can be done either by dividing A by min Aij or by just adding 1. 
# The first approach is kind of cheating because the scaling automatically makes the error small
# If min Aij is not 1, then init error can be LARGE 

#2. If all xj coordinates are updated in parallel, then comparable to scipy (update_v_parllel and update_X_paralle)

#3. If coordinatewise update, then scaling down by n is better than not scaling. 

#4. Our theoreitcal alg in experiment has occasional large jumps in error

#5. The init large error issue is fixed by running a full update step first

#6. x is very very very sparse

#7. TODO m << n case 

using LinearAlgebra, BenchmarkTools, Distributions

const MultivariateDistribution{S<:ValueSupport} = Distribution{Multivariate,S}

const DiscreteMultivariateDistribution   = Distribution{Multivariate, Discrete}
const ContinuousMultivariateDistribution = Distribution{Multivariate, Continuous}

function init_all(eps, n)
    ybar = zeros(n) 
    ykm = zeros(n)  
    y = zeros(n) 
    ktotal = Int(ceil(1/sqrt(eps))) #note that this is just an approx ktotal
    ak = 1/(n-1) 
    Ak = 1/(n-1)
    A_sum_ai_xibar = 0 #A\sum_i ai bar(x_{i-1})
    v = zeros(n) #vector with same length as x, used to obtain x 
    return xbar, xkm, x, ktotal, ak, Ak, A_sum_ai_xibar, v
end

function update_v(jk, pjk, A, y, v, ak) 
    v[jk]+= (ak/pjk)*(A'*(y) - 1)[jk] #scaling the step (making it larger)  to "compensate" for seq vs par
    return v
end

#function update_v_smallstep(jk, pjk, A, y, v, ak) 
#    v[jk]+= (ak)*((A').dot(y) - 1)[jk] #scaling the step (making it larger)  to "compensate" for seq vs par
#    return v
#end
    
#function update_v_parallel(A, y, v, ak) 
#    v+= ak*((A').dot(y) - 1)
#    return v
#end

function compute_scaling(A)
    scaling_vector = -1 ./sum(abs2.(A),dims=2)
    return scaling_vector
end
    
function update_x(v, jk, xkm, scaling) 
    x = xkm 
    x[jk]= min(np.max((scaling*v[jk], 0)),-scaling)
    return x      
end
    
#function update_x_parallel(v, svec)
#    # x= np.max((svec*v, 0))
#    x = np.where(svec*v>0, svec*v, 0)
#    return x
#end

#function update_xbar(x, xkm, ak, akp)
#    xbar = x + (ak/akp)*(x - xkm)
#    return xbar 
#end

function update_ak_Ak(Ak, ak)
    akp = .5* (1 + np.sqrt(1 + 4*Ak))
    Ak = Ak + akp 
    return Ak, akp 
end

function update_y(A_sum_ai_yibar, A, ak, Ak, ybar)
    A_sum_ai_xibar+= A.dot(ak*ybar) #A \sum_{i = 1}^{k} ai xbar_{i-1}
    y_temp = (1/Ak)*A_sum_ai_xibar 
    y = max(0, y_temp) #np.where(x_temp>0, x_temp, 0)
    return y, A_sum_ai_xibar
end

# Oct 31: rescaling

function remove_col1(A,b)
    s=A*b'
    B=A[vec(s.>0),:]
    s=s[vec(s.>0)]
    A=B./s
    return A, B, s
end


In [ ]:
# Main loop
eps = 0.000001 
n = 200 # input dimension 
m = 20 # Number of data
   
#This is A is random positive and objective is max_x ||Ax||^2/2-1^T x case.
#A = np.random.rand(200, 20) 
    
# b can also be random and negative. m>>n.
b=rand(1,m)-repeat([0.3],1,m)
A=rand(n,m)
#A=A+1
(A,B,s)=remove_col1(A,b)

# Also need to scale b
xsum = 0 

#%%
(ybar, ykm, y, ktotal, ak, Ak, A_sum_ai_xibar, v) = init_all(eps, n)
scaling_vector = compute_scaling(A)
our_result = zeros(ktotal)
x_norm = zeros(ktotal)
v_norm = zeros(ktotal)
xsum_norm = zeros(ktotal)
# Xmatrix = zeros((n, ktotal))
Akarray = zeros(ktotal)
# ktotal = 1
Akarray[1]=Ak
#%%      

for k in 1:ktotal 
                # update y 
        
        # sample jk from multinomial distribution
        randomseed=rand(Multinomial(1, ones(n)/n),1)
        jk = findall(vec(randomseed.==1)) # 
        pjk = 1/n
    
    
        (x, A_sum_ai_xibar) = update_x(A_sum_ai_xibar, A, ak, Ak, ybar) ### this step
        
        #if (k==1):
        #    v = update_v_parallel(A, y, v, ak)
        #    x = update_x_parallel(v, scaling_vector)
        #else: 
#%%         #%% 

        #update v parallelly 
        v = update_v_parallel(A, y, v, ak)
        # # update v 
        #v = update_v(jk, pjk, A, y, v, ak)
        #v_norm[k]  = norm(v, 2)       
        
        # update x
        #x = update_x(v, jk, xkm, scaling_vector[jk])

#%%         
        #update x parallelly 
        x = update_x_parallel(v, scaling_vector)
        #x_norm[k]  = norm(x, 2)
#%%         
#%% 
        

#%%   

        
        #update a 
        (Ak, akp) = update_ak_Ak(Ak, ak) 
        Akarray[k] = Ak
#%%         
        
        #update xbar 
        xbar = update_xbar(x, xkm, ak, akp) 
#%%      
        # update xkm 
        xkm = x 
#%%                        
        #compute running sum 
        xsum+= ak*x
        xsum_norm[k] = sum(xsum==0)
        # update ak  
        ak = akp 
        
        xsol_temp = (1/(Ak-ak))*xsum 
        our_result[k] = norm(B*(xsol_temp/s)-b,2)**2
        #our_result[k] = norm(A.dot(xsol_temp),2)**2/2-sum(xsol_temp)
end